In [5]:
import json
import shutil
from pathlib import Path

In [2]:
tree = Path("..")

In [15]:
delete_me = [".ipynb_checkpoints", "__pycache__", ".virtual_documents"]


def CleanTreeOfTrash(tree):
    for item in tree.iterdir():
        if item.is_dir():
            if any([item.name == s for s in delete_me]):
                print("Removing", item)
                try:
                    shutil.rmtree(str(item))
                except Exception:
                    pass
            else:
                CleanTreeOfTrash(item)


CleanTreeOfTrash(tree)

Removing ../Repo Utilities/.ipynb_checkpoints
Removing ../Gather Data/.ipynb_checkpoints
Removing ../Image/Generative Art/Crawler/Version 1/.ipynb_checkpoints
Removing ../BlueTooth Low Energy/.ipynb_checkpoints


In [14]:
def SecureIPythonOutput(tree):
    for item in tree.iterdir():
        if item.is_dir():
            SecureIPythonOutput(item)
        else:
            if item.suffix == ".ipynb":
                print(f"cleaning {item}")
                data = json.loads(item.read_text())
                for cell in data["cells"]:
                    if cell["cell_type"] == "code":
                        if len(cell["outputs"]) > 0:
                            print(f'\tremoving {len(cell["outputs"])} lines of output')
                            cell["outputs"] = []
                item.write_text(json.dumps(data))


SecureIPythonOutput(tree)

cleaning ../Repo Utilities/Security Sweep.ipynb
	removing 1 lines of output
	removing 1 lines of output
	removing 1 lines of output
cleaning ../Repo Utilities/.ipynb_checkpoints/Security Sweep-checkpoint.ipynb
	removing 1 lines of output
	removing 1 lines of output
	removing 1 lines of output
cleaning ../Gather Data/Scrape GasBuddy.ipynb
cleaning ../Gather Data/Gather Fda DrugShortages.ipynb
cleaning ../Gather Data/.ipynb_checkpoints/Gather Fda DrugShortages-checkpoint.ipynb
cleaning ../Arduino/Probe Read Audioize.ipynb
cleaning ../Games/RuneScape/Daily Advancement Email.ipynb
cleaning ../Games/RuneScape/RS Shop Indexer.ipynb
cleaning ../Games/RuneScape/Character Data.ipynb
cleaning ../Games/RuneScape/Daily Character Snapshot Cron.ipynb
cleaning ../Games/RuneScape/Burial Armor Exp Check.ipynb
cleaning ../Games/RuneScape/Market Data.ipynb
cleaning ../Memory Exploration/Sandbox.ipynb
cleaning ../Memory Exploration/Evil/Evil Notebook.ipynb
cleaning ../RemoteZMQDictionary/Monitor Dictionar

In [4]:
from collections import defaultdict

skip_folders = [".git"]


class Result:
    def __init__(self):
        self.keys = []

    def __repr__(self):
        return ", ".join(self.keys)


def IterTree(tree):
    discovered = defaultdict(Result)
    for item in tree.iterdir():
        if item.is_dir():
            if item.name in skip_folders:
                continue
            discovered.update(IterTree(item))
        else:
            try:
                data = item.read_text().lower()
                for search in [
                    "pw",
                    "password",
                    "token",
                    "cred",
                    "auth",
                    "bearer",
                    "user",
                    "client_secret",
                ]:
                    if search in data:
                        discovered[item].keys.append(search)
            except UnicodeDecodeError:
                pass
            except Exception as e:
                print(type(e), e)
                print()
    return dict(discovered)


results = IterTree(tree)
results

{PosixPath('../Repo Utilities/Security Sweep.ipynb'): pw, password, token, cred, auth, bearer, user, client_secret,
 PosixPath('../Games/RuneScape/Daily Advancement Email.ipynb'): user,
 PosixPath('../Games/RuneScape/.env'): user,
 PosixPath('../Games/RuneScape/Character Data.ipynb'): user,
 PosixPath('../Games/RuneScape/Daily Character Snapshot Cron.ipynb'): user,
 PosixPath('../WebView 3/Templates/Login.html'): password, auth, user,
 PosixPath('../WebView 3/Templates/Error.html'): auth,
 PosixPath('../WebView 3/Templates/partials/frame.html'): auth, user,
 PosixPath('../WebView 3/lastRender.html'): auth, user,
 PosixPath('../WebView 3/pages/loginPage.py'): password, user,
 PosixPath('../Memory Exploration/Sandbox.ipynb'): token,
 PosixPath('../Android/Android Clicker Sandbox.ipynb'): pw, auth, user,
 PosixPath('../Android/Beholder/wikiwords.lst'): pw, password, token, cred, auth, bearer, user,
 PosixPath('../Android/Beholder/rs3_dev.ipynb'): pw, user,
 PosixPath('../Android/Beholder/